In [1]:
pip install sentence_transformers

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 645kB 6.0MB/s 
     |████████████████████████████████| 3.8MB 13.8MB/s 
     |████████████████████████████████| 890kB 38.9MB/s 
     |████████████████████████████████| 1.1MB 58.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=dad0f4698e6d42b11bfaa4f7c0fa98e78a1f57258286bc879ba767612c48fbcd
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e02d3b6fa2d86dbb2263b7c195c7bc525ca627873421e207c1f55359cc7bff43
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [0]:
from sentence_transformers import SentenceTransformer
import scipy.spatial as sc
import pandas as pd
import re

In [3]:
embedder = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

100%|██████████| 1.31G/1.31G [02:07<00:00, 10.3MB/s]


In [4]:
df = pd.read_json(r'/content/drive/My Drive/Colab Notebooks/30-04-2020 GoogleSearchResults.json')
df.drop_duplicates(subset ='snippet', keep='first', inplace= True)
df.reset_index(inplace=True)
df.drop(columns='index')

,title,domain,url,snippet,keywords,timestamp,Label
0,Molded interconnect device - Wikipedia,en.wikipedia.org,https://en.wikipedia.org/wiki/Molded_interconn...,,molded interconnect device MID electronics,2020-04-30 11:48:00,Blacklisted
1,3D-MID Technology with Laser Direct Structurin...,www.lpkf.com,https://www.lpkf.com/en/industries-technologie...,Highly integrated electronic 3D devices. With ...,molded interconnect device MID electronics,2020-04-30 11:48:00,Good
2,What are molded interconnect devices? - EDN,www.edn.com,https://www.edn.com/design/pc-board/4427506/Wh...,"Jan 24, 2014 - Molded interconnect devices (MI...",molded interconnect device MID electronics,2020-04-30 11:48:00,Good
3,Molded Interconnect Devices - LPKF Laser & Ele...,www.lpkfusa.com,https://www.lpkfusa.com/products/mid/,LPKF Laser Direct Structuring (LDS) Molded Int...,molded interconnect device MID electronics,2020-04-30 11:48:00,Good
4,MID / LDS Technology – Molex,www.molex.com,https://www.molex.com/molex/capabilities/mid__...,Molded interconnect devices (MIDs) and laser d...,molded interconnect device MID electronics,2020-04-30 11:48:00,Good
...,...,...,...,...,...,...,...
1407,Newsletter 1 - IMEC Projects,projects.imec.be,https://projects.imec.be/terasel/sites/project...,electronics and sensor circuits to be seamless...,molded interconnect device MID circuit printed...,2020-04-30 12:37:00,Good
1408,Manufacturers of Electronic Component Parts - ...,octopart.com,https://octopart.com/manufacturers,Octopart helps you discover manufacturers and ...,molded interconnect device MID film insert mol...,2020-04-30 12:40:00,Good
1409,A circular economy for plastics - HBM4EU,www.hbm4eu.eu,https://www.hbm4eu.eu/wp-content/uploads/2019/...,Koopmans (Plastics Innovation Competence Centr...,molded interconnect device MID film insert mol...,2020-04-30 12:40:00,Good
1410,Assemblies Suppliers on Jobshop.com,www.jobshop.com,https://www.jobshop.com/Assemblies.aspx,Plastics - Injection Molding Plastics - Insert...,molded interconnect device MID film insert mol...,2020-04-30 12:40:00,Blacklisted


In [0]:
sentence = []
tidysentence=[]
similarity = ['cosine', 'correlation',  'euclidean', 'jaccard', ]

In [0]:
for i in range(len(df)):
    st = df['snippet'][i]
    tidysentence.append(st)
    st = re.sub('[^a-zA-z0-9\s]','',st)
    st = '.'.join(st.rsplit('\n'))
    sentence.append(st)

In [7]:
len(sentence)

1412

In [0]:
sentence_embb = embedder.encode(sentence)

In [0]:
query = ["molded interconnect device MID electronics Two-shot molding"]
query_embb = embedder.encode(query[0])
Closest = 5
for i in similarity:
  res=[]
  score = []

  for que, que_embb in zip(query,query_embb):
    distance = sc.distance.cdist([que_embb], sentence_embb,str(i))[0]
    result = zip(range(len(distance)), distance)
    result = sorted(result, key=lambda x: x[1])
    
    for ide, leng in result[0:Closest]:
      score.append(1-leng)
      Parag = sentence[ide]
      #print(score)
      #print(Parag)
      res.append(df.iloc[ide])
  csv_result =[]
  csv_result = pd.DataFrame(res)
  final_res = pd.DataFrame()
  final_res["Title"] = csv_result['title']
  final_res["Domain"] = csv_result["domain"]
  final_res["Url"] = csv_result["url"]
  final_res["Snippet"] = csv_result["snippet"]
  final_res[str(i)+'_Scores'] = pd.Series(score, index=final_res.index)
  final_res.to_csv('/content/drive/My Drive/Colab Notebooks/Output/roberta-large-nli-stsb-mean-tokens/'+query[0]+'with_'+str(i)+'_similarity.csv')





In [20]:
res

[index                                                        0
 title                   Molded interconnect device - Wikipedia
 domain                                        en.wikipedia.org
 url          https://en.wikipedia.org/wiki/Molded_interconn...
 snippet                                                       
 keywords            molded interconnect device MID electronics
 timestamp                                  2020-04-30 11:48:00
 Label                                              Blacklisted
 Name: 0, dtype: object,
 index                                                        1
 title        3D-MID Technology with Laser Direct Structurin...
 domain                                            www.lpkf.com
 url          https://www.lpkf.com/en/industries-technologie...
 snippet      Highly integrated electronic 3D devices. With ...
 keywords            molded interconnect device MID electronics
 timestamp                                  2020-04-30 11:48:00
 Label         

In [11]:
res

[index                                                        0
 title                   Molded interconnect device - Wikipedia
 domain                                        en.wikipedia.org
 url          https://en.wikipedia.org/wiki/Molded_interconn...
 snippet                                                       
 keywords            molded interconnect device MID electronics
 timestamp                                  2020-04-30 11:48:00
 Label                                              Blacklisted
 Name: 0, dtype: object,
 index                                                        1
 title        3D-MID Technology with Laser Direct Structurin...
 domain                                            www.lpkf.com
 url          https://www.lpkf.com/en/industries-technologie...
 snippet      Highly integrated electronic 3D devices. With ...
 keywords            molded interconnect device MID electronics
 timestamp                                  2020-04-30 11:48:00
 Label         

In [57]:
res

[[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0)]]

In [41]:
res

[index                                                        0
 title                   Molded interconnect device - Wikipedia
 domain                                        en.wikipedia.org
 url          https://en.wikipedia.org/wiki/Molded_interconn...
 snippet                                                       
 keywords            molded interconnect device MID electronics
 timestamp                                  2020-04-30 11:48:00
 Label                                              Blacklisted
 Name: 0, dtype: object,
 index                                                        1
 title        3D-MID Technology with Laser Direct Structurin...
 domain                                            www.lpkf.com
 url          https://www.lpkf.com/en/industries-technologie...
 snippet      Highly integrated electronic 3D devices. With ...
 keywords            molded interconnect device MID electronics
 timestamp                                  2020-04-30 11:48:00
 Label         